In [ ]:
FILE = "input.txt"

In [ ]:
from collections import namedtuple
from typing import Literal, get_args, cast
from dataclasses import dataclass


Symbol = Literal["|", "-", "L", "J", "7", "F", ".", "S"]

class Point(namedtuple("Point", ["y", "x"])):
    def __add__(self, other):
        return Point(self.y + other.y, self.x + other.x)


ADJACENT_COORDINATES = [
    Point(-1,  0),
    Point( 1,  0),
    Point( 0, -1),
    Point( 0,  1)
]

CONNECTS_TO: dict[Symbol, list[Point]] = {
    "|": [Point(-1,  0), Point(1,  0)],
    "-": [Point( 0, -1), Point(0,  1)],
    "L": [Point(-1,  0), Point(0,  1)],
    "J": [Point(-1,  0), Point(0, -1)],
    "7": [Point( 1,  0), Point(0, -1)],
    "F": [Point( 1,  0), Point(0,  1)],
    ".": [],
    "S": [], # handled separately
}

STARTING_CHAR = "S"

@dataclass(eq=True, frozen=True)
class MapTile:
    symbol: Symbol
    point: Point
    
    def __str__(self) -> str:
        return str(self.symbol)

    def __repr__(self) -> str:
        return f"MapTile({self.symbol} @ (y={self.point.y}, x={self.point.x}))"


class Map(list[list[MapTile]]):
    def __init__(self, iterable):
        self.min_y = 0
        self.max_y = len(iterable) - 1
        self.min_x = 0
        self.max_x = len(iterable[0]) - 1
        super().__init__(iterable)
    
    def all_tiles(self):
        for row in self:
            for tile in row:
                yield tile

    def __getitem__(self, index):
        if isinstance(index, Point):
            y, x = index
            return self[y][x]
        return super().__getitem__(index)
    
    def point_valid(self, point: Point) -> bool:
        return (self.min_y <= point.y <= self.max_y) and (self.min_x <= point.x <= self.max_x)
    
    def get_adjacent(self, tile: MapTile) -> list[MapTile]:
        point = tile.point
        adjacent = [point + adj for adj in ADJACENT_COORDINATES]
        adjacent = [point for point in adjacent if self.point_valid(point)]
        adjacent_tiles = [self[point] for point in adjacent]
        return adjacent_tiles
    
    def possibly_connected(self, tile: MapTile) -> set[MapTile]:
        point = tile.point
        connected_coords = [point + coords for coords in CONNECTS_TO[tile.symbol]] # type: ignore
        connected_coords = [point for point in connected_coords if self.point_valid(point)]
        return set(self[point] for point in connected_coords)
        
    def get_connected(self, tile: MapTile) -> list[MapTile]:
        connected = []
        tile_connects_to = self.possibly_connected(tile)
        for other_tile in tile_connects_to:
            other_connected_to = self.possibly_connected(other_tile)
            if tile in other_connected_to:
                connected.append(other_tile)
        return connected

In [ ]:
def ensure_is_symbol(char: str) -> Symbol:
    possible_values = get_args(Symbol)
    if char in possible_values:
        return cast(Symbol, char)
    raise ValueError(f"Wrong character: {char}. Should be one of {possible_values}")

def parse_map(data: str) -> Map:
    return Map(
        [
            [
                MapTile(ensure_is_symbol(char), Point(y, x)) for x, char in enumerate(list(row))
            ]
            for y, row in enumerate(data.splitlines())
        ]
    )

def print_map(map: Map):
    for row in map:
        print("[", *row, "]")

In [ ]:
with open(f"10/{FILE}") as f:
    data = f.read()
map = parse_map(data)

In [ ]:
print_map(map)

## Coordinates
Coordinates are assumed to be (y, x) with (0, 0) in the bottom left, y increasing down, and x increasing right.

In [ ]:
def find_starting_point(map: Map) -> MapTile:
    for tile in map.all_tiles():
        if tile.symbol == STARTING_CHAR:
            return tile
    raise ValueError(f"Incorrect map, no starting char: {STARTING_CHAR}")

In [ ]:
starting_point = find_starting_point(map)
adjacent_to_start = map.get_adjacent(starting_point)
connected_to_start = []
for tile in adjacent_to_start:
    possibly_connected = map.possibly_connected(tile)
    if starting_point in possibly_connected:
        connected_to_start.append(tile)
connected_to_start

In [ ]:
visited = {starting_point: 0}
to_visit = [(1, connected) for connected in connected_to_start]

while len(to_visit) != 0:
    (distance, tile), *to_visit = to_visit
    if tile not in visited:
        visited[tile] = distance
        connected = map.get_connected(tile)
        for connected_tile in connected:
            if connected_tile not in visited:
                to_visit.append((distance+1, connected_tile))

In [ ]:
def print_visited(map: Map, visited: dict[MapTile, int]):
    for row in map:
        vis = []
        for tile in row:
            if tile in visited:
                vis.append(visited[tile])
            else:
                vis.append(".")
        print("[", *vis, "]")

In [ ]:
visited

In [ ]:
print_visited(map, visited)

In [ ]:
max(visited.values())